In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Dropout
from tensorflow.keras.metrics import Recall,Precision
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50, ResNet152
from tensorflow.keras.layers import Input,concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.callbacks import ModelCheckpoint


In [3]:
df=pd.read_csv(r"/Users/priyankapalaniselvam/Downloads/classifier_one_data/full_df.csv")


In [4]:
if 'ID' in df.columns:
    df=df.drop('ID',axis=1)

In [5]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['labels'] = label_encoder.fit_transform(df['labels'])
df['labels'] = df['labels'].astype(str)
left_images=df[['Left-Fundus','labels']]
Right_images=df[['Right-Fundus','labels']]

left_images.columns=['image','target']
Right_images.columns=['image','target']


In [6]:
combined_df=pd.concat([left_images,Right_images])
combined_df = combined_df.dropna(subset=['image'])
combined_df

,image,target
0,0_left.jpg,6
1,1_left.jpg,6
2,2_left.jpg,2
3,4_left.jpg,2
4,5_left.jpg,2
...,...,...
6387,4686_right.jpg,2
6388,4688_right.jpg,2
6389,4689_right.jpg,2
6390,4690_right.jpg,2


In [7]:
train_df,test_df=train_test_split(combined_df,test_size=0.2,random_state=42)
train_df.head()

,image,target
5328,3007_right.jpg,6
108,128_right.jpg,7
4023,950_left.jpg,6
3189,4686_right.jpg,2
1494,2231_right.jpg,1


In [8]:
datagen=ImageDataGenerator(rescale=1./255)

In [9]:
train_generator=datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='/Users/priyankapalaniselvam/Downloads/classifier_one_data/ODIR-5K/ODIR-5K/Training Images',
    x_col='image',
    y_col='target',
    subset="training",
    class_mode="categorical",
    target_size=(224,224),
    batch_size=16
)


Found 10227 validated image filenames belonging to 8 classes.


In [10]:
validation_generator = datagen.flow_from_dataframe(
    dataframe=test_df,
    directory='/Users/priyankapalaniselvam/Downloads/classifier_one_data/ODIR-5K/ODIR-5K/Training Images',
    x_col="image",
    y_col="target",
    # subset="validation",
    class_mode="categorical",
    target_size=(224, 224),
    batch_size=32
    )

Found 2557 validated image filenames belonging to 8 classes.


In [11]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [12]:
input_shape = (224, 224, 3)

input_layer = Input(shape=input_shape)
num_classes = 8
base_model = EfficientNetB7(include_top=False, weights='imagenet', input_tensor=input_layer)
x = base_model.output
dense1 = GlobalAveragePooling2D()(x)
dense2 = Dense(256, activation='relu')(dense1)
dense3 = Dense(128, activation='relu')(dense2)
dense4 = Dense(64, activation='relu')(dense3)

output_layer = Dense(8, activation='softmax')(dense4)
model = Model(inputs=input_layer, outputs=output_layer)


In [13]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [14]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min', verbose=1)


In [17]:
history = model.fit(train_generator, epochs=5, validation_data=validation_generator)


Epoch 1/5
640/640 ━━━━━━━━━━━━━━━━━━━━ 14908s 23s/step - accuracy: 0.7442 - loss: 0.7143 - val_accuracy: 0.6574 - val_loss: 1.0190
Epoch 2/5
640/640 ━━━━━━━━━━━━━━━━━━━━ 23821s 37s/step - accuracy: 0.7536 - loss: 0.6621 - val_accuracy: 0.6903 - val_loss: 0.9749
Epoch 3/5
640/640 ━━━━━━━━━━━━━━━━━━━━ 12357s 19s/step - accuracy: 0.8018 - loss: 0.5277 - val_accuracy: 0.6989 - val_loss: 1.0040
Epoch 4/5
640/640 ━━━━━━━━━━━━━━━━━━━━ 9909s 15s/step - accuracy: 0.8192 - loss: 0.4571 - val_accuracy: 0.6930 - val_loss: 1.1324
Epoch 5/5
640/640 ━━━━━━━━━━━━━━━━━━━━ 9753s 15s/step - accuracy: 0.8399 - loss: 0.3873 - val_accuracy: 0.6785 - val_loss: 1.1498


In [18]:
model.save('eff_b7.h5')

In [22]:
model.save('my_model.keras')

In [23]:
model.save_weights('efficientnet_b7.weights.h5')
